#Initialization

In [13]:
from google.colab import drive
import numpy as np
import os
import cv2
import tensorflow as tf
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import BatchNormalization, Conv2D, Softmax, Reshape
from tensorflow.keras.models import Model
import shutil
import zipfile

drive.mount('/content/drive')

num_classes = 2
ARCHIVE_PATH = '/content/drive/MyDrive/data24/victims_corner/archives'
CSV_PATH = '/content/drive/MyDrive/data24/victims_corner/csv'

LOCAL_CSV_PATH = '/content/data/csv'
INPUT_IMAGE_PATH = '/content/data/input_images'
MODEL_INPUT_IMAGE_PATH = '/content/data/model_input_images'
TARGET_OUTPUT_PATH = '/content/data/targets'

MODEL_OUTPUT_PATH = '/content/drive/MyDrive/victims_v3.tflite'

os.makedirs(LOCAL_CSV_PATH, exist_ok=True)
os.makedirs(INPUT_IMAGE_PATH, exist_ok=True)
os.makedirs(MODEL_INPUT_IMAGE_PATH, exist_ok=True)
os.makedirs(TARGET_OUTPUT_PATH, exist_ok=True)

def remove_quotation_marks(x: str):
  return x.replace('"', '').strip()

input_height = 96
input_width = 96
input_shape = (input_height, input_width, 1)

target_width = 12
target_height = 12

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Load dataset

##Unpack archives, copy CSVs

In [3]:
for archive in os.listdir(ARCHIVE_PATH):
  print(f'Unpacking {archive}...')
  with zipfile.ZipFile(os.path.join(ARCHIVE_PATH, archive)) as zip_file:
    for member in zip_file.namelist():
      filename = os.path.basename(member)

      if not filename:
        continue

      source = zip_file.open(member)
      target = open(os.path.join(INPUT_IMAGE_PATH, filename), 'wb')
      with source, target:
        shutil.copyfileobj(source, target)

for csv in os.listdir(CSV_PATH):
  print(f'Copying {csv}...')
  with open(os.path.join(CSV_PATH, csv), 'r') as f:
    f2 = open(os.path.join(LOCAL_CSV_PATH, csv), 'w')
    f2.write(f.read())
    f2.close()

Unpacking victims_corner_2023_12_28.zip...
Unpacking victims_2023_12_27.zip...
Unpacking 20240223.zip...
Unpacking 20240224.zip...
Unpacking victims_2024_02_28.zip...
Copying victims_2023_12_27.csv...
Copying victims_2023_12_28.csv...
Copying victims_2024_02_23-export.csv...
Copying victims_28_02_2024-export.csv...


##Resize input for model

In [ ]:
for filename in os.listdir(INPUT_IMAGE_PATH):
  img = cv2.imread(os.path.join(INPUT_IMAGE_PATH, filename))
  if img is None or img.shape != (240, 320, 3):
    print(f'{filename} is wrong size.')
    continue
  img = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), (input_width, input_height))
  cv2.imwrite(os.path.join(MODEL_INPUT_IMAGE_PATH, filename), img)

##Generate targets

In [19]:
targets = {}
empty_target = np.full((target_width, target_height, 3), [128, -127, -127], dtype=np.int8)

#Build model

In [ ]:
num_classes_with_background = num_classes + 1

width, height, input_channels = input_shape

if width != height:
  raise Exception(f"Only square inputs are supperted; not {input_shape}")

mobile_net_v2 = MobileNetV2(input_shape=input_shape,
                            alpha=alpha,
                            include_top=True)
for layer in mobile_net_v2.layers:
  if type(layer) == BatchNormalization:
    layer.momentum = 0.9

cut_point = mobile_net_v2.get_layer('block_5_expand_relu')

model = Conv2D(filters=32, kernel_size=1, strides=1,
                activation='relu', name='head')(cut_point.output)
logits = Conv2D(filters=num_classes_with_background, kernel_size=1, strides=1,
                activation=None, name='logits')(model)

model = Model(inputs=mobile_net_v2.input, outputs=logits)

model_output_shape = model.layers[-1].output.shape
_batch, width, height, num_classes = model_output_shape
if width != height:
  raise Exception(f"Only square outputs are supperted; not {model_output_shape}")
output_width_height = width

prefetch_policy = tf.data.experimental.AUTOTUNE

train_segmentation_dataset = as_segmentation(train_dataset, True)
validation_segmentation_dataset = as_segmentation(validation_dataset, False)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=learning_rate))

model.fit(train_segmentation_dataset, validation_data=validation_segmentation_dataset,
          epochs=num_epochs, verbose=0)

softmax_layer = Softmax()(model.layers[-1].output)
model = Model(model.input, softmax_layer)

#Train

In [ ]:
model = train(num_classes=2,
              learning_rate=0.001,
              num_epochs=20,
              alpha=0.35,
              object_weight=100,
              train_dataset=train_dataset,
              validation_dataset=validation_dataset,
              best_model_path='',
              input_shape=input_shape,
              batch_size=32)

#Test

In [ ]:
from google.colab import drive

import os
import cv2
import numpy as np
import sys
import glob
import random
import importlib.util
from tensorflow.lite.python.interpreter import Interpreter

import matplotlib
import matplotlib.pyplot as plt
import time

drive.mount('/content/drive')

imgpath = '/content/drive/MyDrive/data24/victims_corner/manual_test'
modelpath = '/content/drive/MyDrive/allgood2.lite'

num_test_images = 8

# Grab filenames of all images in test folder
images = glob.glob(imgpath + '/*.jpg') + glob.glob(imgpath + '/*.JPG') + glob.glob(imgpath + '/*.png') + glob.glob(imgpath + '/*.bmp')

# Load the label map into memory
labels = ['living', 'dead']

# Load the Tensorflow Lite model into memory
interpreter = Interpreter(model_path=modelpath)
interpreter.allocate_tensors()

# Get model details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]

print(input_details)

print(f"Model expects {width}x{height} image")

float_input = (input_details[0]['dtype'] == np.float32)

input_mean = 127.5
input_std = 127.5

# Randomly select test images
images_to_test = random.sample(images, num_test_images)

# Loop over every image and perform detection
for image_path in images_to_test:

    # Load image and resize to expected shape [1xHxWx3]
    image = cv2.imread(image_path)
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    imH, imW, _ = image.shape
    image_resized = cv2.resize(image_gray, (width, height))
    input_data = np.expand_dims(np.expand_dims(image_resized, axis=0), axis=3)
    print(input_data.shape)

    # Normalize pixel values if using a floating model (i.e. if model is non-quantized)
    if float_input:
        input_data = (np.float32(input_data) - input_mean) / input_std
    else:
      # Convert uint8 to int8
      input_data = np.int8(input_data - input_mean)

    # Perform the actual detection by running the model with the image as input
    interpreter.set_tensor(input_details[0]['index'],input_data)

    start_time = time.time()
    interpreter.invoke()

    end_time = time.time()

    # Retrieve detection results
    boxes = interpreter.get_tensor(output_details[0]['index'])[0,:,:,:] # Bounding box coordinates of detected objects
    classes = interpreter.get_tensor(output_details[0]['index'])[0] # Class index of detected objects
    scores = interpreter.get_tensor(output_details[0]['index'])[0] # Confidence of detected objects

    output_width = boxes.shape[1]
    output_height = boxes.shape[0]
    input_width = image.shape[1]
    input_height = image.shape[0]

    boxes = boxes + 128

    print("Time: ", end_time - start_time)

    detections = []

    contours_living, _ = cv2.findContours(cv2.threshold(np.uint8(boxes[:,:,2]), 180, 255, cv2.THRESH_BINARY)[1], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours_dead, _ = cv2.findContours(cv2.threshold(np.uint8(boxes[:,:,1]), 180, 255, cv2.THRESH_BINARY)[1], cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    for arr, label in [(contours_living, True), (contours_dead, False)]:
      for i in range(len(arr)):
        x, y, w, h = cv2.boundingRect(arr[i])
        detections.append((x * input_width / output_width, y * input_height / output_height, label))

    for detection in detections:
      color = (255, 0, 0)
      if detection[2]:
        color = (0, 255, 0)

      cv2.circle(image, (int(detection[0]), int(detection[1])), 5, color, 3)

      print(detection)


    plt.figure(figsize=(12, 16))
    plt.imshow(image)

    # Loop over all detections and draw detection box if confidence is above minimum threshold
    """for i in range(len(scores)):
        if ((scores[i] > min_conf) and (scores[i] <= 1.0)):

            # Get bounding box coordinates and draw box
            # Interpreter can return coordinates that are outside of image dimensions, need to force them to be within image using max() and min()
            ymin = int(max(1,(boxes[i][0] * imH)))
            xmin = int(max(1,(boxes[i][1] * imW)))
            ymax = int(min(imH,(boxes[i][2] * imH)))
            xmax = int(min(imW,(boxes[i][3] * imW)))

            cv2.rectangle(image, (xmin,ymin), (xmax,ymax), (10, 255, 0), 1)

            # Draw label
            object_name = labels[int(classes[i])] # Look up object name from "labels" array using class index
            label = '%s: %d%%' % (object_name, int(scores[i]*100)) # Example: 'person: 72%'
            labelSize, baseLine = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.4, 1) # Get font size
            label_ymin = max(ymin, labelSize[1] + 10) # Make sure not to draw label too close to top of window
            cv2.rectangle(image, (xmin, label_ymin-labelSize[1]-10), (xmin+labelSize[0], label_ymin+baseLine-10), (255, 255, 255), cv2.FILLED) # Draw white box to put label text in
            cv2.putText(image, label, (xmin, label_ymin-7), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 0), 1) # Draw label text

            detections.append([object_name, scores[i], xmin, ymin, xmax, ymax])"""


Output hidden; open in https://colab.research.google.com to view.